In [56]:
import pandas as pd
from datetime import date, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [95]:
df = pd.read_csv('./bydatestore_xwordinfo.csv',  na_filter=False) # There are some clues with "NULL" as the answer.

In [96]:
df.head()

,year,month,day,weekday,clue,answer,index
0,1993,11,21,6,Site of the Shandog Peninsula,YELLOWSEA,1-ACROSS
1,1993,11,21,6,"""Closer to Fine"" singers",INDIGOGIRLS,5-ACROSS
2,1993,11,21,6,Capital near Lake Tuz,ANKARA,11-ACROSS
3,1993,11,21,6,MasterCard rival,OPTIMA,17-ACROSS
4,1993,11,21,6,Petrarch works,SONNETS,19-ACROSS


In [97]:
len(df)

875542

## Checks

In [98]:
# Ensure there are no duplicates
df[df.duplicated(keep=False)]

,year,month,day,weekday,clue,answer,index


In [99]:
grouped = df.groupby(['year','month','day'])
print(len(grouped))

# Check all dates covered
start_date = date(1993, 11, 21)
end_date = date(2022, 8, 1)
for single_date in daterange(start_date, end_date):
    if (single_date.year, single_date.month, single_date.day) not in grouped.groups.keys():
        print(single_date.strftime('%m/%d/%y')) 

10480


## Fix mistake

In [78]:
def fix_missing_downs(gdf):
    oneacrosses = gdf[gdf['index'] == '1-ACROSS'].index
    if len(oneacrosses) == 2:
        downstart = oneacrosses[1]
        gdf.loc[downstart:,'index'] = gdf.loc[downstart:,'index'].apply(lambda x: x.replace('ACROSS', 'DOWN'))
    return gdf

In [79]:
res_df = grouped.apply(fix_missing_downs)

In [80]:
len(res_df)

875542

In [84]:
res_df[(res_df['year'] == 2008) & (res_df['month'] == 1) & (res_df['day'] == 1)]

,year,month,day,weekday,clue,answer,index
814425,2008,1,1,1,"""Mamma Mia"" group",ABBA,1-ACROSS
814426,2008,1,1,1,Play chauffeur,DRIVE,5-ACROSS
814427,2008,1,1,1,Money to help one through a tight spot,LOAN,10-ACROSS
814428,2008,1,1,1,Either of two directing brothers,COEN,14-ACROSS
814429,2008,1,1,1,"All gone, as dinner",EATEN,15-ACROSS
...,...,...,...,...,...,...,...
814498,2008,1,1,1,"A couple of chips, maybe",ANTE,54-DOWN
814499,2008,1,1,1,Injury reminder,SCAR,55-DOWN
814500,2008,1,1,1,Purchases for a shindig,KEGS,56-DOWN
814501,2008,1,1,1,"___ 9000, sci-fi computer",HAL,58-DOWN


In [89]:
tres_df = res_df.sort_values(['year','month','day']).reset_index()

In [90]:
tres_df.head()

,level_0,year,month,day,weekday,clue,answer,index
0,0,1993,11,21,6,Site of the Shandog Peninsula,YELLOWSEA,1-ACROSS
1,1,1993,11,21,6,"""Closer to Fine"" singers",INDIGOGIRLS,5-ACROSS
2,2,1993,11,21,6,Capital near Lake Tuz,ANKARA,11-ACROSS
3,3,1993,11,21,6,MasterCard rival,OPTIMA,17-ACROSS
4,4,1993,11,21,6,Petrarch works,SONNETS,19-ACROSS


In [91]:
tres_df[(tres_df['year'] == 2008) & (tres_df['month'] == 1) & (tres_df['day'] == 1)]

,level_0,year,month,day,weekday,clue,answer,index
432083,814425,2008,1,1,1,"""Mamma Mia"" group",ABBA,1-ACROSS
432084,814426,2008,1,1,1,Play chauffeur,DRIVE,5-ACROSS
432085,814427,2008,1,1,1,Money to help one through a tight spot,LOAN,10-ACROSS
432086,814428,2008,1,1,1,Either of two directing brothers,COEN,14-ACROSS
432087,814429,2008,1,1,1,"All gone, as dinner",EATEN,15-ACROSS
...,...,...,...,...,...,...,...,...
432156,814498,2008,1,1,1,"A couple of chips, maybe",ANTE,54-DOWN
432157,814499,2008,1,1,1,Injury reminder,SCAR,55-DOWN
432158,814500,2008,1,1,1,Purchases for a shindig,KEGS,56-DOWN
432159,814501,2008,1,1,1,"___ 9000, sci-fi computer",HAL,58-DOWN


In [93]:
tres_df.drop('level_0', axis=1, inplace=True)
tres_df.head()

,year,month,day,weekday,clue,answer,index
0,1993,11,21,6,Site of the Shandog Peninsula,YELLOWSEA,1-ACROSS
1,1993,11,21,6,"""Closer to Fine"" singers",INDIGOGIRLS,5-ACROSS
2,1993,11,21,6,Capital near Lake Tuz,ANKARA,11-ACROSS
3,1993,11,21,6,MasterCard rival,OPTIMA,17-ACROSS
4,1993,11,21,6,Petrarch works,SONNETS,19-ACROSS


In [94]:
tres_df.to_csv('./bydatestore_xwordinfo.csv', index=False)

## Prepare Answer Counts and Ranks

In [100]:
count_df = pd.DataFrame(df.groupby('answer')['clue'].count())
count_df.columns = ['answer_count']
count_df['answer_rank'] = count_df['answer_count'].rank(method='min', ascending=False)

In [101]:
count_df.sort_values('answer_count', ascending=False)

,answer_count,answer_rank
answer,,
ERA,659,1.0
AREA,550,2.0
ERE,532,3.0
ONE,524,4.0
ELI,509,5.0
...,...,...
JACOBASTOR,1,62178.0
JACKY,1,62178.0
JACKWEBB,1,62178.0


In [102]:
big_df = pd.merge(
    df,
    count_df,
    how="left",
    on="answer",
    copy=True,
    indicator=False,
    validate=None,
)
big_df.head()

,year,month,day,weekday,clue,answer,index,answer_count,answer_rank
0,1993,11,21,6,Site of the Shandog Peninsula,YELLOWSEA,1-ACROSS,6,22799.0
1,1993,11,21,6,"""Closer to Fine"" singers",INDIGOGIRLS,5-ACROSS,5,25810.0
2,1993,11,21,6,Capital near Lake Tuz,ANKARA,11-ACROSS,35,5248.0
3,1993,11,21,6,MasterCard rival,OPTIMA,17-ACROSS,16,10860.0
4,1993,11,21,6,Petrarch works,SONNETS,19-ACROSS,10,15790.0


In [103]:
big_df.tail()

,year,month,day,weekday,clue,answer,index,answer_count,answer_rank
875537,2022,7,31,6,Inits. on the road,MPH,108-DOWN,56,3009.0
875538,2022,7,31,6,Inits. on the road,AAA,109-DOWN,203,256.0
875539,2022,7,31,6,"Darkroom item, in brief",NEG,110-DOWN,52,3295.0
875540,2022,7,31,6,Personal ad abbr.,SWF,111-DOWN,13,12868.0
875541,2022,7,31,6,No-win situation,TIE,112-DOWN,240,146.0


In [104]:
display(big_df.dtypes)
big_df[big_df['answer_count'].isna()]

year              int64
month             int64
day               int64
weekday           int64
clue             object
answer           object
index            object
answer_count      int64
answer_rank     float64
dtype: object

,year,month,day,weekday,clue,answer,index,answer_count,answer_rank


In [106]:
big_df['answer_count'] = big_df['answer_count'].astype(int)
big_df['answer_rank'] = big_df['answer_rank'].astype(int)

In [107]:
big_df.head()

,year,month,day,weekday,clue,answer,index,answer_count,answer_rank
0,1993,11,21,6,Site of the Shandog Peninsula,YELLOWSEA,1-ACROSS,6,22799
1,1993,11,21,6,"""Closer to Fine"" singers",INDIGOGIRLS,5-ACROSS,5,25810
2,1993,11,21,6,Capital near Lake Tuz,ANKARA,11-ACROSS,35,5248
3,1993,11,21,6,MasterCard rival,OPTIMA,17-ACROSS,16,10860
4,1993,11,21,6,Petrarch works,SONNETS,19-ACROSS,10,15790


In [108]:
big_df.to_csv('./bydatestore_xwordinfo_processed.csv', index=False)